# Creating Bike and Pedestrian Road Links

For environment see: https://github.com/gboeing/osmnx 

    conda config --prepend channels conda-forge
    
    conda create -n ox --strict-channel-priority osmnx
    
* also include nb_conda, nb_conda_kernels, and ipykernel

In [1]:
import geopandas as gpd
import osmnx
import pandas as pd
import networkx as nx

In [ ]:
#update the attributes retrieved from OSM in settings.py
useful_tags_path = ['bridge', 'tunnel', 'oneway', 'lanes', 'ref', 'name',
                    'highway', 'maxspeed', 'service', 'access', 'area',
                    'landuse', 'width', 'est_width', 'junction', 
                    'surface', 'smoothness','cycleway', 'footway', 'sidewalk']
osmnx.utils.config(useful_tags_way = useful_tags_path)

utn = osmnx.settings.useful_tags_node
oxna = osmnx.settings.osm_xml_node_attrs
oxnt = osmnx.settings.osm_xml_node_tags
utw = osmnx.settings.useful_tags_way
oxwa = osmnx.settings.osm_xml_way_attrs
oxwt = osmnx.settings.osm_xml_way_tags
utn = list(set(utn + oxna + oxnt))
utw = list(set(utw + oxwa + oxwt))
osmnx.config(all_oneway=True, useful_tags_node=utn, useful_tags_way=utw)

#grab from OSM the bike and pedestrian networks for the MPO
#WARNING: Takes 48 hours to run (for both bike and ped together)
places = ['New Shoreham, Rhode Island','Jamestown, Rhode Island', 'Prudence Island, Rhode Island', 'Rhode Island', 
          'Cape Cod, Massachusetts', 'Chappaquiddick Island, Massachusetts','Monomoy Island, Massachusetts',
          'Long Island, Massachusetts', 'Spectacle Island, Massachusetts', 'Thompson Island, Massachusetts',
          'Little Brewster Island, Massachusetts', 'Great Brewster Island, Massachusetts', 
          'Shag Rocks Island, Massachusetts', 'Calf Island, Massachusetts', 'Little Calf Island, Massachusetts',
          'Middle Brewster Island, Massachusetts', 'Outer Brewster Island, Massachusetts',
          'Moon Island, Massachusetts', 'Rainsford Island, Massachusetts', 'Gallops Island, Massachusetts',
          'Georges Island, Massachusetts', 'Lovells Island, Massachusetts', 'Peddocks Island, Massachusetts',
          'Nantucket, Massachusetts', "Martha's Vineyard, Massachusetts", 'Naushon Island, Massachusetts',
          'Nashawena Island, Massachusetts', 'Gosnold, Massachusetts','Provincetown, Massachusetts',
          'Massachusetts', 'Greenville, New Hampshire', 'Lyndeborough, New Hampshire', 'New Ipswich, New Hampshire',
         'Epping, New Hampshire', 'Exeter, New Hampshire', 'Epping, New Hampshire', 'Greenland, New Hampshire',
         'Hampstead, New Hampshire', 'Kingston, New Hampshire', 'New Castle, New Hampshire', 'Manchester, New Hampshire',
         'Pelham, New Hampshire', 'Temple, New Hampshire','Amherst, New Hampshire', 'Bedford, New Hampshire',
         'Brookline, New Hampshire', 'Goffstown, New Hampshire', 'Hollis, New Hampshire', 'Hudson, New Hampshire',
         'Litchfield, New Hampshire', 'Mason, New Hampshire', 'Merrimack, New Hampshire', 'Milford, New Hampshire',
         'Mont Vernon, New Hampshire', 'Nashua, New Hampshire', 'New Boston, New Hampshire', 'Sharon, New Hampshire',
         'Wilton, New Hampshire', 'Newfields, New Hampshire', 'Newington, New Hampshire', 'Newton, New Hampshire',
         'North Hampton, New Hampshire', 'Plaistow, New Hampshire', 'Portsmouth, New Hampshire', 'Rye, New Hampshire',
         'Salem, New Hampshire','Sandown, New Hampshire', 'Stratham, New Hampshire', 'Brentwood, New Hampshire',
         'Fremont, New Hampshire', 'Rindge, New Hampshire', 'Atkinson, New Hampshire', 'Auburn, New Hampshire',
         'Candia, New Hampshire', 'Chester, New Hampshire', 'Danville, New Hampshire','Derry, New Hampshire',
         'East Kingston, New Hampshire','Kensington, New Hampshire', 'Raymond, New Hampshire','Seabrook, New Hampshire',
         'Windham, New Hampshire', 'Hampton Falls, New Hampshire', 'Londonderry, New Hampshire',
         'South Hampton, New Hampshire', 'Hampton, New Hampshire', 'Hooksett, New Hampshire']

count = 0
for place in places:
    try:
        bike_net = osmnx.graph_from_place(place, network_type='bike')
        ped_net = osmnx.graph_from_place(place, network_type='walk')
        if count > 0:
            full_bike_net = nx.compose(full_bike_net, bike_net)
            full_ped_net = nx.compose(full_ped_net, ped_net)
        else:
            full_bike_net = bike_net
            full_ped_net = ped_net
        count+=1
    except:
        print(place)
        


In [ ]:
#osmnx.io.save_graph_shapefile(full_bike_net, 'C:/Users/phoebe/Bike_Ped_Skims_OSM/Bike2')
osmnx.io.save_graph_shapefile(full_bike_net, 'M:/JupyterHome/JupyterNotebooks/Ped_Bike_Skims_OSM/Ped_Bike_Skims_OSM/Bike2/PartNH')

In [ ]:
#osmnx.io.save_graph_shapefile(full_ped_net, 'C:/Users/phoebe/Bike_Ped_Skims_OSM/Ped2')
osmnx.io.save_graph_shapefile(full_ped_net, 'M:/JupyterHome/JupyterNotebooks/Ped_Bike_Skims_OSM/Ped_Bike_Skims_OSM/Ped2/PartNH')